In [1]:
import pandas as pd
import numpy as np

In [2]:
movie_row = pd.read_excel('Data/Input/CF1.xls', sheet_name='movie-row')

In [3]:
movie_row.head()

,Unnamed: 0,1648,5136,918,2824,3867,860,3712,2968,3525,...,3556,5261,2492,5062,2486,4942,2267,4809,3853,2288
0,11: Star Wars: Episode IV - A New Hope (1977),NaN,4.5,5.0,4.5,4.0,4.0,NaN,5.0,4.0,...,4.0,NaN,4.5,4.0,3.5,NaN,NaN,NaN,NaN,NaN
1,12: Finding Nemo (2003),NaN,5.0,5.0,NaN,4.0,4.0,4.5,4.5,4.0,...,4.0,NaN,3.5,4.0,2.0,3.5,NaN,NaN,NaN,3.5
2,13: Forrest Gump (1994),NaN,5.0,4.5,5.0,4.5,4.5,NaN,5.0,4.5,...,4.0,5.0,3.5,4.5,4.5,4.0,3.5,4.5,3.5,3.5
3,14: American Beauty (1999),NaN,4.0,NaN,NaN,NaN,NaN,4.5,2.0,3.5,...,4.0,NaN,3.5,4.5,3.5,4.0,NaN,3.5,NaN,NaN
4,22: Pirates of the Caribbean: The Curse of the...,4.0,5.0,3.0,4.5,4.0,2.5,NaN,5.0,3.0,...,3.0,1.5,4.0,4.0,2.5,3.5,NaN,5.0,NaN,3.5


In [4]:
# User-user pearson correlation matrix, 1 to -1. > 0 means positively correlated and vice-versa.
uu_similarity_matrix = movie_row.corr()

In [5]:
print(uu_similarity_matrix.shape)
uu_similarity_matrix.head()

(25, 25)


,1648,5136,918,2824,3867,860,3712,2968,3525,4323,...,3556,5261,2492,5062,2486,4942,2267,4809,3853,2288
1648,1.000000,0.402980,-0.142206,0.517620,0.300200,0.480537,-0.312412,0.383348,0.092775,0.098191,...,-0.191988,0.493008,0.360644,0.551089,0.002544,0.116653,-0.429183,0.394371,-0.304422,0.245048
5136,0.402980,1.000000,0.118979,0.057916,0.341734,0.241377,0.131398,0.206695,0.360056,0.033642,...,0.488607,0.328120,0.422236,0.226635,0.305803,0.037769,0.240728,0.411676,0.189234,0.390067
918,-0.142206,0.118979,1.000000,-0.317063,0.294558,0.468333,0.092037,-0.045854,0.367568,-0.035394,...,0.373226,0.470972,0.069956,-0.054762,0.133812,0.015169,-0.273096,0.082528,0.667168,0.119162
2824,0.517620,0.057916,-0.317063,1.000000,-0.060913,-0.008066,0.462910,0.214760,0.169907,0.119350,...,-0.201275,0.228341,0.238700,0.259660,0.247097,0.149247,-0.361466,0.474974,-0.262073,0.166999
3867,0.300200,0.341734,0.294558,-0.060913,1.000000,0.282497,0.400275,0.264249,0.125193,-0.333602,...,0.174085,0.297977,0.476683,0.293868,0.438992,-0.162818,-0.295966,0.054518,0.464110,0.379856


In [6]:
# Identify the top 5 neighbors (the users with the 5 largest, positive correlations) for users
def get_topN(uu_similarity_matrix, userId, n=5):
    # added to have total 5 output, rather than 4, as excluding itself from the list
    return uu_similarity_matrix.loc[userId].nlargest(n+1).iloc[1:]

In [7]:
get_topN(uu_similarity_matrix, userId=3712)

2824    0.462910
3867    0.400275
5062    0.247693
442     0.227130
3853    0.193660
Name: 3712, dtype: float64

In [27]:
def get_weighted_avg(movie, userId):
    weighted_rating_sum = 0
    weighted_sum = 0
    # neighbors is
    neighbors = get_topN(uu_similarity_matrix, userId)
    for uId, w in zip(neighbors.index, neighbors.values):
        # skip the below steps where rating doesn't exist
        if np.isnan(movie[uId]):
            continue
        #print(movie[uId], w)
        weighted_rating_sum += movie[uId] * w
        weighted_sum += w
        
    if weighted_sum == 0:
        return 0
    return weighted_rating_sum/weighted_sum

In [26]:
user_3867_rslt = movie_row.apply(get_weighted_avg, axis=1, userId=3712)

4.5 0.4629100498862757
4.0 0.40027450425381644
4.0 0.24769327229404767
3.0 0.2271298649307886
4.0 0.40027450425381644
4.0 0.24769327229404767
5.0 0.4629100498862757
4.5 0.40027450425381644
4.5 0.24769327229404767
3.0 0.2271298649307886
3.5 0.19365960183726966
4.5 0.24769327229404767
4.0 0.2271298649307886
4.5 0.4629100498862757
4.0 0.40027450425381644
4.0 0.24769327229404767
1.0 0.2271298649307886
4.0 0.4629100498862757
3.0 0.40027450425381644
4.0 0.24769327229404767
5.0 0.2271298649307886
5.0 0.19365960183726966
4.0 0.24769327229404767
3.0 0.2271298649307886
5.0 0.19365960183726966
2.0 0.2271298649307886
4.0 0.19365960183726966
5.0 0.4629100498862757
4.0 0.24769327229404767
4.0 0.2271298649307886
4.5 0.19365960183726966
4.5 0.40027450425381644
4.0 0.24769327229404767
5.0 0.2271298649307886
4.0 0.19365960183726966
5.0 0.4629100498862757
4.0 0.40027450425381644
4.0 0.24769327229404767
4.0 0.2271298649307886
3.0 0.19365960183726966
4.5 0.24769327229404767
5.0 0.2271298649307886
5.0 0.462

In [21]:
user_3867_rslt.sort_values(ascending=False)[:3]

50    4.855924
21    4.622564
15    4.616916
dtype: float64

In [22]:
movie_row.iloc[user_3867_rslt.sort_values(ascending=False)[:3].index,0]

50                               603: The Matrix (1999)
21                          155: The Dark Knight (2008)
15    121: The Lord of the Rings: The Two Towers (2002)
Name: Unnamed: 0, dtype: object